In [34]:
import numpy as np

from nupack import Complex as NupackComplex
from nupack import Model as NupackModel
from nupack import ComplexSet as NupackComplexSet
from nupack import Strand as NupackStrand
from nupack import SetSpec as NupackSetSpec
from nupack import complex_analysis as nupack_complex_analysis
from nupack import PairsMatrix as NupackPairsMatrix


import dsd.vienna_nupack as dv
import dsd.constraints as dc


# constants
DOMAIN_LENGTH = 15
DOMAIN_POOL: dc.DomainPool = dc.DomainPool('DOMAIN_POOL', DOMAIN_LENGTH)
TEMPERATURE = 37.0
NUPACK_MODEL = NupackModel(material='dna', celsius=TEMPERATURE)
NUMBER_OF_TRIALS = 100

#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##----->
#                     |||||  |||||
#                    <-----##-----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*^   b*
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['b*', 'a*'], name='bot strand')

for s in [top_strand, bot_strand]:
    for d in s.domains:
        d.pool = DOMAIN_POOL

base_pair_probabilities = [0] * NUMBER_OF_TRIALS
for i in range(NUMBER_OF_TRIALS):
    ## Assign random DNA sequence
    #
    strand_dna_sequence = dv.random_dna_seq(2*DOMAIN_LENGTH)
    # Assigning to top_strand automatically reassigns bot_strand
    top_strand.assign_dna(strand_dna_sequence)
    
    ## Initialize NUPACK NupackComplexSet arguments
    #
    nupack_strands = [NupackStrand(strand.sequence(), name=strand.name) for strand in (top_strand, bot_strand)] #
    nupack_complex: NupackComplex = NupackComplex(nupack_strands)
    nupack_complex_set = NupackComplexSet(nupack_strands, complexes=(nupack_complex,))
    
    # Call NUPACK complex_analysis
    nupack_complex_analysis_result = nupack_complex_analysis(nupack_complex_set, compute=['pairs'], model=NUPACK_MODEL)
    pairs: NupackPairsMatrix = nupack_complex_analysis_result[nupack_complex].pairs
    nupack_complex_result: np.ndarray = pairs.to_array()
    prob = nupack_complex_result[14][45]
    base_pair_probabilities[i] = prob

base_pair_probabilities= np.array(base_pair_probabilities)
print(base_pair_probabilities)
print(np.std(base_pair_probabilities))
print(np.mean(base_pair_probabilities))



[0.99902794 0.99983311 0.99694816 0.99769139 0.99769385 0.99983292
 0.99940965 0.99620452 0.99938795 0.99947088 0.99043261 0.99524616
 0.99626152 0.99874704 0.99769235 0.99982895 0.9926188  0.99237538
 0.99968282 0.98197729 0.99072573 0.99546661 0.99756475 0.9952322
 0.99983302 0.99902506 0.99900561 0.99078411 0.99072432 0.99972154
 0.99968167 0.99929612 0.99875956 0.99929573 0.99940847 0.99625913
 0.99617966 0.99769346 0.99696145 0.99694831 0.99253944 0.99757047
 0.99814466 0.99947988 0.9923273  0.99815304 0.9994007  0.99626475
 0.99043832 0.99510158 0.99235759 0.99072758 0.99258854 0.99759956
 0.99968345 0.99619067 0.99972402 0.99900668 0.99769144 0.99509401
 0.99068595 0.99423284 0.99869581 0.99693922 0.99760845 0.99875396
 0.99899659 0.9998331  0.98213151 0.99619734 0.99046235 0.99925205
 0.99074422 0.99765169 0.99761296 0.99760573 0.99626476 0.98510791
 0.99871749 0.9950985  0.99544614 0.99545577 0.99696289 0.99258904
 0.99696722 0.99900668 0.99769577 0.99972109 0.99509803 0.98198